<a href="https://colab.research.google.com/github/Sandhiya1305/Fake_News_Prediction/blob/main/Fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

# Reading the data
data = pd.read_csv("news.csv")
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
	title.append(data['title'][x])
	text.append(data['text'][x])
	labels.append(data['label'][x])

In [ ]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
	sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [ ]:
import numpy as np

embeddings_index = {}
with open('glove.6B.50d.txt', encoding='utf-8') as f:  # Specify UTF-8 encoding
    for line in f:
        values = line.strip().split()  # Strip whitespace for consistency
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except ValueError:
            print(f"Skipping line: {line.strip()}")  # Print problematic lines for debugging
            continue
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

Skipping line: hoÃ ng -0.021476 -0.46788 -0.57489 0.25384 -0.83486 -1.2622 0.74499 0.41843 0.15076 -0.3762 0.31586 1.0779 0.26601 -0.84584 -0.21545 -0.30827 -1.0899 -0.53668 2.0235 1.3613 -0.9779 -0.80162 0.68561 -0.18649 -0.5215 0.8261 -0.1142 -0.37842 -0.70749 -0.082986 -0.54944 -0.25965 0.39062 1.385 0.35108 0.75802 0.064603 1.0825 -0.023899 -0.59102 0.16235 -0.25107 0.20679 -0.12571 -0.018305 -0.91191 -0.27518 -1.3942 0.9766 -0.11565
Skipping line: vis-Ã -vis -0.42222 -0.17201 -0.3977 -0.47292 0.1869 -0.27254 1.1996 0.30363 0.026903 -0.25643 0.43997 0.43185 0.11064 0.071013 -1.2559 -0.17438 0.39446 0.17034 0.78422 0.10057 -0.25143 -0.54215 -0.36984 0.44356 0.14029 0.50077 -0.41179 -0.20118 -0.070628 0.67134 -0.37876 0.23519 0.66533 -0.04729 -0.13505 0.0080215 -0.22851 -0.11957 -1.2082 0.24583 0.22041 -0.70773 0.25961 0.24909 -0.52742 -0.43718 -0.06806 0.56763 0.3998 0.06423
Skipping line: thÃ nh -0.149 -0.53056 -0.34744 -1.1326 -0.26678 -0.75053 1.7498 0.79765 -0.59799 0.14863 -0.6

In [ ]:
model = tf.keras.Sequential([
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
							input_length=max_length, weights=[
								embeddings_matrix],
							trainable=False),
	tf.keras.layers.Dropout(0.2),  #prevent overfitting
	tf.keras.layers.Conv1D(64, 5, activation='relu'),
	tf.keras.layers.MaxPooling1D(pool_size=4),
	tf.keras.layers.LSTM(64),
	tf.keras.layers.Dense(1, activation='sigmoid')    #o/p probability
])
model.compile(loss='binary_crossentropy',
			optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 54, 50)            377600    
                                                                 
 dropout_2 (Dropout)         (None, 54, 50)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 50, 64)            16064     
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 12, 64)            0         
 g1D)                                                            
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                      

In [ ]:
num_epochs = 3

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
					epochs=num_epochs,
					validation_data=(testing_padded,
									testing_labels),
					verbose=2)   #verbose-showing progress after each epochs


Train on 2700 samples, validate on 300 samples
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 3s - loss: 0.6663 - accuracy: 0.5741 - val_loss: 0.6050 - val_accuracy: 0.6800 - 3s/epoch - 938us/sample
Epoch 2/3
2700/2700 - 2s - loss: 0.5837 - accuracy: 0.6937 - val_loss: 0.5427 - val_accuracy: 0.7167 - 2s/epoch - 620us/sample
Epoch 3/3
2700/2700 - 2s - loss: 0.5434 - accuracy: 0.7374 - val_loss: 0.5159 - val_accuracy: 0.7100 - 2s/epoch - 584us/sample


In [ ]:
# sample text to check if fake or not
X = "Obama makes the right call to tough it out in Afghanistan"
# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
						padding=padding_type,
						truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):   #true-1(>0.5 is closer to 1)so news is true
	print("This news is True")
else:
	print("This news is false")

This news is True
